# **Aplicación de modelo de segmentación de clientes**

## **Preparación de los datos**

In [1]:
# Importarr todas las librerías necesarias para el proyecto

import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.impute import SimpleImputer
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA

from sklearn.metrics import (
    silhouette_score,
    davies_bouldin_score,
    calinski_harabasz_score
)
from sklearn.metrics import adjusted_rand_score

### *Sales*

In [2]:
# Importar archivo de "Sales"

sales = pd.read_excel('Sales.xlsx')

sales.head()    

,Row_ID,Order_ID,Order_Date,Ship_Date,Ship_Mode,Customer_ID,Country/Region,City,State/Province,Postal_Code,...,Discount,Profit,Productos.Category,Productos.Sub-Category,unit_list_price,profit_per_unit,% utilidad,Dias de envio,Discount Band,Discount Band Order
0,1,US-2020-103800,2020-01-03,2020-01-07,Standard Class,DP-13000,United States,Houston,Texas,77095,...,0.2,5.5512,Office Supplies,Paper,10.28,2.7756,0.27,4,1–20%,1
1,2,US-2020-112326,2020-01-04,2020-01-08,Standard Class,PO-19195,United States,Naperville,Illinois,60540,...,0.8,-5.4870,Office Supplies,Binders,8.85,-2.7435,-0.31,4,71–80%,7
2,3,US-2020-112326,2020-01-04,2020-01-08,Standard Class,PO-19195,United States,Naperville,Illinois,60540,...,0.2,4.2717,Office Supplies,Labels,4.91,1.4239,0.29,4,1–20%,1
3,4,US-2020-112326,2020-01-04,2020-01-08,Standard Class,PO-19195,United States,Naperville,Illinois,60540,...,0.2,-64.7748,Office Supplies,Storage,113.64,-21.5916,-0.19,4,1–20%,1
4,5,US-2020-141817,2020-01-05,2020-01-12,Standard Class,MB-18085,United States,Philadelphia,Pennsylvania,19143,...,0.2,4.8840,Office Supplies,Art,8.14,1.6280,0.20,7,1–20%,1


In [3]:
# Borrar columnas innecesarias

sales = sales.drop(columns=['Discount Band Order', 'Discount Band', 'Dias de envio', 'profit_per_unit' ])  

In [4]:
sales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11737 entries, 0 to 11736
Data columns (total 20 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   Row_ID                  11737 non-null  int64         
 1   Order_ID                11737 non-null  object        
 2   Order_Date              11737 non-null  datetime64[ns]
 3   Ship_Date               11737 non-null  datetime64[ns]
 4   Ship_Mode               11737 non-null  object        
 5   Customer_ID             11737 non-null  object        
 6   Country/Region          11666 non-null  object        
 7   City                    11737 non-null  object        
 8   State/Province          11737 non-null  object        
 9   Postal_Code             11737 non-null  object        
 10  Region                  11737 non-null  object        
 11  Product_ID              11737 non-null  object        
 12  Sales                   11737 non-null  float6

In [5]:
sales.describe()

,Row_ID,Order_Date,Ship_Date,Sales,Quantity,Discount,Profit,unit_list_price,% utilidad
count,11737.000000,11737,11737,11737.000000,11737.000000,11737.000000,11737.000000,11737.000000,11737.000000
mean,5125.508307,2022-05-03 18:51:33.601431040,2022-05-07 12:18:13.158387712,228.085882,3.778734,0.156655,29.074615,75.921766,0.185404
min,1.000000,2020-01-03 00:00:00,2020-01-07 00:00:00,0.444000,1.000000,0.000000,-6599.978000,0.990000,-0.630000
25%,2573.000000,2021-05-23 00:00:00,2021-05-27 00:00:00,17.340000,2.000000,0.000000,1.760800,6.480000,0.060000
50%,5143.000000,2022-07-03 00:00:00,2022-07-04 00:00:00,53.970000,3.000000,0.200000,8.694000,19.980000,0.260000
75%,7668.000000,2023-05-18 00:00:00,2023-05-20 00:00:00,207.350000,5.000000,0.200000,29.245000,73.950000,0.330000
max,10194.000000,2023-12-30 00:00:00,2024-01-05 00:00:00,22638.480000,14.000000,0.800000,8399.976000,7546.160000,0.500000
std,2941.568558,NaN,NaN,621.061889,2.213594,0.206894,235.881124,191.882709,0.232254


### *Returns*

In [6]:
# Importar archivo de "Returns"

returns = pd.read_excel('Returns.xlsx')

returns.head()

,Order_ID,Productos devueltos
0,US-2020-100762,4
1,US-2020-100867,1
2,US-2020-102652,4
3,US-2020-103373,1
4,US-2020-103744,2


In [7]:
returns.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 296 entries, 0 to 295
Data columns (total 2 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Order_ID             296 non-null    object
 1   Productos devueltos  296 non-null    int64 
dtypes: int64(1), object(1)
memory usage: 4.8+ KB


In [8]:
returns.describe()

,Productos devueltos
count,296.000000
mean,2.702703
std,1.899008
min,1.000000
25%,1.000000
50%,2.000000
75%,3.000000
max,14.000000


### *Costumer*

In [9]:
# Importar archivo de "Costumer"

costumer = pd.read_excel('Costumer.xlsx')

costumer.head()

,Customer_ID,Customer_Name,monthly_salary,age,marital_status [1:married/ 0:Single],credit score
0,AA-10315,Alex Avila,6349,32,1,500
1,AA-10375,Allen Armold,5374,26,0,630
2,AA-10480,Andrew Allen,9611,44,1,780
3,AA-10645,Anna Andreadi,7025,43,1,450
4,AB-10015,Aaron Bergman,3331,28,1,490


In [10]:
costumer.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 804 entries, 0 to 803
Data columns (total 6 columns):
 #   Column                                Non-Null Count  Dtype 
---  ------                                --------------  ----- 
 0   Customer_ID                           804 non-null    object
 1   Customer_Name                         804 non-null    object
 2   monthly_salary                        804 non-null    object
 3   age                                   804 non-null    int64 
 4   marital_status [1:married/ 0:Single]  804 non-null    int64 
 5   credit score                          804 non-null    int64 
dtypes: int64(3), object(3)
memory usage: 37.8+ KB


In [11]:
costumer.describe()

,age,marital_status [1:married/ 0:Single],credit score
count,804.000000,804.000000,804.000000
mean,38.415423,0.513682,557.947761
std,10.901532,0.500124,148.992105
min,26.000000,0.000000,300.000000
25%,32.000000,0.000000,420.000000
50%,38.000000,1.000000,560.000000
75%,44.000000,1.000000,690.000000
max,260.000000,1.000000,800.000000


### *Regional manager*

In [12]:
# Importar archivo de "Regional_manager"

rm = pd.read_excel('Regional_manager.xlsx')

rm.head()

,Regional_Manager,Region
0,Chuck Magee,East
1,Fred Suzuki,South
2,Roxanne Rodriguez,Central
3,Sadie Pawthorne,West


In [13]:
rm.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4 entries, 0 to 3
Data columns (total 2 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Regional_Manager  4 non-null      object
 1   Region            4 non-null      object
dtypes: object(2)
memory usage: 196.0+ bytes


In [14]:
rm.describe()

,Regional_Manager,Region
count,4,4
unique,4,4
top,Chuck Magee,East
freq,1,1


### *Product*

In [15]:
# Importar archivo de "product"

product = pd.read_excel('Product.xlsx')

product.head()

,Product_ID,Categoría,Sub-Categoría,Product_Name
0,FUR-BO-10000112,Furniture,Bookcases,"Bush Birmingham Collection Bookcase, Dark Cherry"
1,FUR-BO-10000330,Furniture,Bookcases,"Sauder Camden County Barrister Bookcase, Plank..."
2,FUR-BO-10000362,Furniture,Bookcases,Sauder Inglewood Library Bookcases
3,FUR-BO-10000468,Furniture,Bookcases,O'Sullivan 2-Shelf Heavy-Duty Bookcases
4,FUR-BO-10000711,Furniture,Bookcases,"Hon Metal Bookcases, Gray"


In [16]:
product.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1862 entries, 0 to 1861
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Product_ID     1862 non-null   object
 1   Categoría      1862 non-null   object
 2   Sub-Categoría  1862 non-null   object
 3   Product_Name   1862 non-null   object
dtypes: object(4)
memory usage: 58.3+ KB


## **2.1 Segmentación de clientes**

In [17]:
# ============================================================
# 2.1 Segmentación de clientes 
# - Numeric-only KMeans 
# - Winsorization p1-p99
# - Signed-log profit
# - PCA opcional + tuning hiperparámetros
# - Métricas: Silhouette, DBI, CH, Inertia + Estabilidad (ARI)
# ============================================================



# ---------------------------
# Configuración
# ---------------------------
FILES = {
    "sales": "Sales.xlsx",
    "returns": "Returns.xlsx",
    "customer": "Costumer.xlsx",         
    "rm": "Regional_manager.xlsx",
    "product": "Product.xlsx"
}

RANDOM_STATE = 42

COL = {
    "customer_id": "Customer_ID",
    "order_id": "Order_ID",
    "order_date": "Order_Date",
    "ship_date": "Ship_Date",
    "sales": "Sales",
    "profit": "Profit",
    "discount": "Discount",
    "qty": "Quantity",
    "ship_mode": "Ship_Mode",
    "region": "Region",
    "country": "Country/Region",
    "state": "State/Province",
    "city": "City",
    "product_id": "Product_ID",
    "category": "Productos.Category",
    "subcat": "Productos.Sub-Category",
    "returned_units": "Productos devueltos",   
}

# ---------------------------
# Helpers
# ---------------------------
def to_num(series: pd.Series) -> pd.Series:
    """Converts series to numeric safely."""
    return pd.to_numeric(series, errors="coerce")

def signed_log1p(x: pd.Series) -> pd.Series:
    """Signed log transform: sign(x)*log1p(abs(x)). Keeps negative signal."""
    x = pd.to_numeric(x, errors="coerce")
    return np.sign(x) * np.log1p(np.abs(x))

def winsorize_series(s: pd.Series, p_low=0.01, p_high=0.99) -> pd.Series:
    """Caps series at given percentiles."""
    s = pd.to_numeric(s, errors="coerce")
    lo = s.quantile(p_low)
    hi = s.quantile(p_high)
    return s.clip(lower=lo, upper=hi)

def safe_col(df: pd.DataFrame, colname: str) -> bool:
    return colname in df.columns

# ---------------------------
# 1) Cargar datos
# ---------------------------
sales = pd.read_excel(FILES["sales"])
returns = pd.read_excel(FILES["returns"])
customer = pd.read_excel(FILES["customer"])
rm = pd.read_excel(FILES["rm"])
product = pd.read_excel(FILES["product"])

# ---------------------------
# 2) Limpieza mínima / consistencia
# ---------------------------

# Dedup customer
if safe_col(customer, COL["customer_id"]):
    customer = customer.drop_duplicates(subset=[COL["customer_id"]]).copy()

# Dedup product (si hay duplicados por Product_ID)
if safe_col(product, COL["product_id"]):
    sort_cols = [COL["product_id"]]
    if "Product_Name" in product.columns:
        sort_cols.append("Product_Name")
    product = (
        product.sort_values(sort_cols)
               .drop_duplicates(subset=[COL["product_id"]], keep="first")
               .copy()
    )

# Tipos de datos en sales
for c in [COL["sales"], COL["profit"], COL["discount"], COL["qty"]]:
    if safe_col(sales, c):
        sales[c] = to_num(sales[c])

# Fechas
if safe_col(sales, COL["order_date"]):
    sales[COL["order_date"]] = pd.to_datetime(sales[COL["order_date"]], errors="coerce")
if safe_col(sales, COL["ship_date"]):
    sales[COL["ship_date"]] = pd.to_datetime(sales[COL["ship_date"]], errors="coerce")

# Discount clean
if safe_col(sales, COL["discount"]):
    sales["Discount_clean"] = np.where(sales[COL["discount"]] > 1, sales[COL["discount"]] / 100, sales[COL["discount"]])
else:
    sales["Discount_clean"] = np.nan

# Lead time (si no existe ya)
if safe_col(sales, COL["order_date"]) and safe_col(sales, COL["ship_date"]):
    sales["lead_time_days"] = (sales[COL["ship_date"]] - sales[COL["order_date"]]).dt.days
else:
    sales["lead_time_days"] = np.nan

# Returns -> asegurar 1 fila por orden
# Si returns viene con múltiples filas por Order_ID, se agrupa.
if safe_col(returns, COL["order_id"]):
    if safe_col(returns, COL["returned_units"]):
        ret_order = (returns.groupby(COL["order_id"], as_index=False)
                     .agg(productos_devueltos=(COL["returned_units"], "sum")))
    else:
        # fallback: si no existe 'Productos devueltos', asume flag=1 por fila
        ret_order = (returns.groupby(COL["order_id"], as_index=False)
                     .size().rename(columns={"size": "productos_devueltos"}))
    ret_order["returned_flag"] = (ret_order["productos_devueltos"] > 0).astype(int)
else:
    ret_order = pd.DataFrame(columns=[COL["order_id"], "productos_devueltos", "returned_flag"])

# ---------------------------
# 3) Unir información (df transaccional)
# ---------------------------
df = sales.copy()

# Merge product
if safe_col(df, COL["product_id"]) and safe_col(product, COL["product_id"]):
    # Trae categoría y subcategoría si existen
    prod_cols = [COL["product_id"]]
    for extra in [COL["category"], COL["subcat"], "Product_Name"]:
        if safe_col(product, extra):
            prod_cols.append(extra)
    df = df.merge(product[prod_cols], on=COL["product_id"], how="left")

# Merge customer
if safe_col(df, COL["customer_id"]) and safe_col(customer, COL["customer_id"]):
    df = df.merge(customer, on=COL["customer_id"], how="left")

# Merge returns
if safe_col(df, COL["order_id"]) and safe_col(ret_order, COL["order_id"]):
    df = df.merge(ret_order[[COL["order_id"], "returned_flag", "productos_devueltos"]], on=COL["order_id"], how="left")
else:
    df["returned_flag"] = 0
    df["productos_devueltos"] = 0

df["returned_flag"] = df.get("returned_flag", 0).fillna(0).astype(int)
df["productos_devueltos"] = df.get("productos_devueltos", 0).fillna(0)

# Merge regional manager
if safe_col(df, COL["region"]) and safe_col(rm, COL["region"]):
    df = df.merge(rm, on=COL["region"], how="left")

# ---------------------------
# 4) Tabla a nivel ORDEN
# ---------------------------
group_cols = [COL["customer_id"], COL["order_id"]]
order_tbl = (
    df.groupby(group_cols, as_index=False)
      .agg(
          order_date=(COL["order_date"], "min"),
          ship_date=(COL["ship_date"], "min") if safe_col(df, COL["ship_date"]) else ("Discount_clean", "count"),
          sales=(COL["sales"], "sum"),
          profit=(COL["profit"], "sum"),
          qty=(COL["qty"], "sum"),
          discount_mean=("Discount_clean", "mean"),
          discount_median=("Discount_clean", "median"),
          lead_time_mean=("lead_time_days", "mean"),
          returned_flag=("returned_flag", "max"),
          productos_devueltos=("productos_devueltos", "max"),
      )
)

# Campos “preferidos” por cliente (opcionales para perfil; NO para clustering)
def mode_or_first(s):
    m = s.mode()
    return m.iloc[0] if len(m) else s.iloc[0]

for col in [COL["ship_mode"], COL["region"], COL["country"]]:
    if safe_col(df, col):
        tmp = (df.groupby(group_cols, as_index=False)[col].agg(mode_or_first))
        order_tbl = order_tbl.merge(tmp, on=group_cols, how="left")

# Fecha referencia recencia
max_date = order_tbl["order_date"].max()

# ---------------------------
# 5) Customer 360 (1 fila por cliente)
# ---------------------------
cust = (
    order_tbl.groupby(COL["customer_id"], as_index=False)
             .agg(
                 frequency_orders=(COL["order_id"], "nunique"),
                 monetary_sales=("sales", "sum"),
                 monetary_profit=("profit", "sum"),
                 units_total=("qty", "sum"),
                 avg_discount=("discount_mean", "mean"),
                 med_discount=("discount_median", "mean"),
                 lead_time_mean=("lead_time_mean", "mean"),
                 return_order_rate=("returned_flag", "mean"),
                 avg_products_returned=("productos_devueltos", "mean"),
                 last_order=("order_date", "max"),
                 first_order=("order_date", "min"),
             )
)

cust["aov"] = cust["monetary_sales"] / cust["frequency_orders"].replace(0, np.nan)
cust["margin_pct"] = cust["monetary_profit"] / cust["monetary_sales"].replace(0, np.nan)
cust["units_per_order"] = cust["units_total"] / cust["frequency_orders"].replace(0, np.nan)
cust["recency_days"] = (max_date - cust["last_order"]).dt.days
cust["tenure_days"] = (cust["last_order"] - cust["first_order"]).dt.days

# Diversidad / concentración desde df (si existen Category/Subcat)
if safe_col(df, COL["category"]) or safe_col(df, COL["subcat"]):
    div = {COL["product_id"]: "nunique"}
    if safe_col(df, COL["category"]):
        div[COL["category"]] = "nunique"
    if safe_col(df, COL["subcat"]):
        div[COL["subcat"]] = "nunique"

    diversity = (
        df.groupby(COL["customer_id"], as_index=False)
          .agg(
              product_diversity=(COL["product_id"], "nunique"),
              category_diversity=(COL["category"], "nunique") if safe_col(df, COL["category"]) else (COL["product_id"], "count"),
              subcat_diversity=(COL["subcat"], "nunique") if safe_col(df, COL["subcat"]) else (COL["product_id"], "count"),
          )
    )
else:
    diversity = pd.DataFrame({COL["customer_id"]: cust[COL["customer_id"]],
                             "product_diversity": 0,
                             "category_diversity": 0,
                             "subcat_diversity": 0})

# Top category share
if safe_col(df, COL["category"]):
    cat_sales = (df.groupby([COL["customer_id"], COL["category"]], as_index=False)[COL["sales"]].sum())
    cat_sales["cat_share"] = cat_sales[COL["sales"]] / cat_sales.groupby(COL["customer_id"])[COL["sales"]].transform("sum")
    top_cat = (cat_sales.sort_values([COL["customer_id"], "cat_share"], ascending=[True, False])
                      .drop_duplicates(COL["customer_id"])
                      .rename(columns={"cat_share": "top_category_share"})[[COL["customer_id"], "top_category_share"]])
else:
    top_cat = pd.DataFrame({COL["customer_id"]: cust[COL["customer_id"]], "top_category_share": 1.0})

# Discount behavior por línea
disc_line = (
    df.assign(is_disc=(df["Discount_clean"] > 0).astype(int))
      .groupby(COL["customer_id"], as_index=False)
      .agg(
          pct_lines_discounted=("is_disc", "mean"),
          discount_std=("Discount_clean", "std"),
          discount_p90=("Discount_clean", lambda s: np.nanpercentile(s, 90)),
      )
)
disc_line["discount_std"] = disc_line["discount_std"].fillna(0)

# Unir todo
cust = cust.merge(diversity, on=COL["customer_id"], how="left")
cust = cust.merge(top_cat, on=COL["customer_id"], how="left")
cust = cust.merge(disc_line, on=COL["customer_id"], how="left")

# Traer demografía si existe (ajusta nombres según tu archivo)
# Intentamos columnas típicas; si no existen, no rompe.
demo_cols = [
    "monthly_salary",
    "age",
    "marital_status [1:married/ 0:Single]",
    "credit score"
]
for dc in demo_cols:
    if dc in customer.columns and dc not in cust.columns:
        # ya vienen por merge previo si df los tenía; por seguridad, merge directo desde customer
        pass

if safe_col(customer, COL["customer_id"]):
    keep_demo = [COL["customer_id"]] + [c for c in demo_cols if c in customer.columns]
    cust = cust.merge(customer[keep_demo], on=COL["customer_id"], how="left")

# Defaults seguros
cust["top_category_share"] = cust["top_category_share"].fillna(1.0)
cust["pct_lines_discounted"] = cust["pct_lines_discounted"].fillna(0.0)
cust["discount_std"] = cust["discount_std"].fillna(0.0)
cust["discount_p90"] = cust["discount_p90"].fillna(0.0)
cust["lead_time_mean"] = cust["lead_time_mean"].fillna(cust["lead_time_mean"].median())

# ---------------------------
# 6) Transformaciones para mejorar clustering
# - winsorización en variables sesgadas
# - signed-log profit
# - log en ventas/frecuencia/diversidad
# ---------------------------

# Winsorizar (cap p1-p99) variables típicamente con outliers
for colw in ["monetary_sales", "frequency_orders", "aov", "units_per_order", "units_total"]:
    if colw in cust.columns:
        cust[colw] = winsorize_series(cust[colw], 0.01, 0.99)

# Profit: mantener negativos con signed log (mejora robustez)
cust["profit_signed_log"] = signed_log1p(cust["monetary_profit"])

# Logs para heavy tails (evita duplicar: usaremos logs en lugar de raws donde aplique)
cust["sales_log"] = np.log1p(cust["monetary_sales"].clip(lower=0))
cust["freq_log"] = np.log1p(cust["frequency_orders"].clip(lower=0))
cust["proddiv_log"] = np.log1p(cust["product_diversity"].clip(lower=0))
cust["subcatdiv_log"] = np.log1p(cust["subcat_diversity"].clip(lower=0))

# ---------------------------
# 7) Dataset final NUMÉRICO (para KMeans)
# (Reducimos redundancia: preferimos logs + ratios)
# ---------------------------
num_features = [
    "recency_days",
    "tenure_days",
    "freq_log",
    "sales_log",
    "profit_signed_log",
    "margin_pct",
    "aov",
    "units_per_order",
    "avg_discount",
    "med_discount",
    "discount_std",
    "discount_p90",
    "pct_lines_discounted",
    "lead_time_mean",
    "return_order_rate",
    "avg_products_returned",
    "proddiv_log",
    "subcatdiv_log",
    "category_diversity",
    "top_category_share",
]

# Demografía si existe
for dc in demo_cols:
    if dc in cust.columns:
        num_features.append(dc)

# Coerción numérica
for c in num_features:
    cust[c] = pd.to_numeric(cust[c], errors="coerce")

X = cust[num_features].copy()

# Preprocess numérico: imputar + escalar robusto
preprocess = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", RobustScaler())
])

X_scaled = preprocess.fit_transform(X)

# ---------------------------
# 8) Tuning hiperparámetros + métricas adicionales + estabilidad
# ---------------------------

def evaluate_config(Xmat, k, pca_var=None, init="k-means++", n_init=50, seed=42):
    # PCA opcional
    Xuse = Xmat
    pca_model = None
    if pca_var is not None:
        pca_model = PCA(n_components=pca_var, random_state=seed)
        Xuse = pca_model.fit_transform(Xmat)

    km = KMeans(
        n_clusters=k,
        init=init,
        n_init=n_init,
        max_iter=500,
        random_state=seed
    )
    labels = km.fit_predict(Xuse)

    # Métricas internas
    sil = silhouette_score(Xuse, labels)
    dbi = davies_bouldin_score(Xuse, labels)
    ch = calinski_harabasz_score(Xuse, labels)
    inertia = km.inertia_

    return labels, sil, dbi, ch, inertia, pca_model

def stability_ari(Xmat, k, pca_var=None, init="k-means++", n_init=50, base_seed=42, seeds=(1, 7, 21, 99, 123)):
    # Base
    base_labels, *_ = evaluate_config(Xmat, k, pca_var=pca_var, init=init, n_init=n_init, seed=base_seed)

    aris = []
    for s in seeds:
        lbl, *_ = evaluate_config(Xmat, k, pca_var=pca_var, init=init, n_init=n_init, seed=s)
        aris.append(adjusted_rand_score(base_labels, lbl))
    return float(np.mean(aris)), float(np.std(aris))

# Grid (mantener razonable para tiempo)
k_range = range(2, 11)
pca_options = [None, 0.90, 0.95]          # None = sin PCA
init_options = ["k-means++"]              # puedes agregar "random" si quieres
n_init_options = [20, 50]

rows = []
for pca_var in pca_options:
    for init in init_options:
        for n_init in n_init_options:
            for k in k_range:
                labels, sil, dbi, ch, inertia, _ = evaluate_config(
                    X_scaled, k, pca_var=pca_var, init=init, n_init=n_init, seed=RANDOM_STATE
                )
                ari_mean, ari_std = stability_ari(
                    X_scaled, k, pca_var=pca_var, init=init, n_init=n_init, base_seed=RANDOM_STATE
                )

                rows.append({
                    "k": k,
                    "pca_var": "None" if pca_var is None else pca_var,
                    "init": init,
                    "n_init": n_init,
                    "silhouette": sil,
                    "davies_bouldin": dbi,
                    "calinski_harabasz": ch,
                    "inertia": inertia,
                    "stability_ari_mean": ari_mean,
                    "stability_ari_std": ari_std
                })

results = pd.DataFrame(rows)

# Ranking sugerido (multi-criterio):
# - maximizar silhouette
# - minimizar davies_bouldin
# - maximizar calinski_harabasz
# - maximizar estabilidad (ARI)
results_sorted = results.sort_values(
    by=["silhouette", "stability_ari_mean", "davies_bouldin", "calinski_harabasz"],
    ascending=[False, False, True, False]
).reset_index(drop=True)

print("\nTop 10 configuraciones:")
print(results_sorted.head(10))

# Guardar tabla de selección (para anexar a presentación/justificación)
results_sorted.to_csv("model_selection_results.csv", index=False)
print("\nGuardado: model_selection_results.csv")

# Seleccionar la mejor configuración
best = results_sorted.iloc[0].to_dict()
best_k = int(best["k"])
best_pca = None if best["pca_var"] == "None" else float(best["pca_var"])
best_init = best["init"]
best_n_init = int(best["n_init"])

print("\nMejor configuración elegida:")
print(best)

# ---------------------------
# 9) Entrenar modelo final con mejor configuración
# ---------------------------
final_labels, sil, dbi, ch, inertia, pca_model = evaluate_config(
    X_scaled, best_k, pca_var=best_pca, init=best_init, n_init=best_n_init, seed=RANDOM_STATE
)

cust["cluster"] = final_labels

print("\nMétricas del modelo final:")
print({
    "k": best_k,
    "pca_var": best["pca_var"],
    "silhouette": sil,
    "davies_bouldin": dbi,
    "calinski_harabasz": ch,
    "inertia": inertia
})

# ---------------------------
# 10) Perfilamiento de clusters (tabla ejecutiva)
# ---------------------------
profile = (
    cust.groupby("cluster")
        .agg(
            customers=(COL["customer_id"], "nunique"),
            sales=("monetary_sales", "sum"),
            profit=("monetary_profit", "sum"),
            avg_margin=("margin_pct", "mean"),
            avg_aov=("aov", "mean"),
            avg_freq=("frequency_orders", "mean"),
            avg_recency=("recency_days", "mean"),
            avg_discount=("avg_discount", "mean"),
            pct_lines_discounted=("pct_lines_discounted", "mean"),
            return_rate=("return_order_rate", "mean"),
            lead_time=("lead_time_mean", "mean")
        )
        .reset_index()
)

profile["sales_share"] = profile["sales"] / profile["sales"].sum()
profile["profit_share"] = profile["profit"] / profile["profit"].sum()
profile["margin_weighted"] = profile["profit"] / profile["sales"].replace(0, np.nan)

print("\nPerfil de clusters (ordenado por sales_share):")
print(profile.sort_values("sales_share", ascending=False))

# ---------------------------
# 11) Exportables
# ---------------------------
# output con features + cluster (para Power BI)
out_cols = [COL["customer_id"], "cluster"] + num_features
cust[out_cols].to_csv("customer_segmentation_output.csv", index=False)

profile.to_csv("cluster_profile_summary.csv", index=False)

print("\nGuardados:")
print("- customer_segmentation_output.csv")
print("- cluster_profile_summary.csv")



Top 10 configuraciones:
   k pca_var       init  n_init  silhouette  davies_bouldin  \
0  2     0.9  k-means++      20    0.295570        1.581067   
1  2     0.9  k-means++      50    0.295570        1.581067   
2  3     0.9  k-means++      20    0.294895        1.421296   
3  3     0.9  k-means++      50    0.294895        1.421296   
4  2    0.95  k-means++      20    0.279683        1.648649   
5  2    0.95  k-means++      50    0.279683        1.648649   
6  3    0.95  k-means++      20    0.278000        1.487754   
7  3    0.95  k-means++      50    0.278000        1.487754   
8  2    None  k-means++      20    0.270127        1.695127   
9  2    None  k-means++      50    0.270127        1.695127   

   calinski_harabasz       inertia  stability_ari_mean  stability_ari_std  
0         186.674953  12396.032206            1.000000           0.000000  
1         186.674953  12396.032206            1.000000           0.000000  
2         174.279296  10647.882839            1.00000

## **2.2. Aplicación con ML**

In [18]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np

from sklearn.model_selection import TimeSeriesSplit, RandomizedSearchCV
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, RobustScaler
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import HistGradientBoostingClassifier

from sklearn.metrics import (
    roc_auc_score, average_precision_score,
    confusion_matrix, classification_report,
    f1_score, precision_score, recall_score
)

# ============================================================
# 0) Config
# ============================================================
FILES = {
    "sales": "Sales.xlsx",
    "returns": "Returns.xlsx",
    "customer": "Costumer.xlsx",
    "rm": "Regional_manager.xlsx",
    "product": "Product.xlsx",
}

COL = {
    "customer_id": "Customer_ID",
    "order_id": "Order_ID",
    "order_date": "Order_Date",
    "sales": "Sales",
    "profit": "Profit",
    "discount": "Discount",
    "qty": "Quantity",
    "ship_mode": "Ship_Mode",
    "region": "Region",
    "country": "Country/Region",
    "state": "State/Province",
    "city": "City",
    "product_id": "Product_ID",
    "category": "Productos.Category",
    "subcat": "Productos.Sub-Category",
    "returned_units": "Productos devueltos"
}

RANDOM_STATE = 42

def safe_ohe():
    try:
        return OneHotEncoder(handle_unknown="ignore", sparse_output=False)
    except TypeError:
        return OneHotEncoder(handle_unknown="ignore", sparse=False)

def _norm(s: str) -> str:
    return str(s).lower().replace("-", " ").replace("_", " ").strip()

def find_category_col(df: pd.DataFrame):
    cols = df.columns.tolist()
    exact = {"category", "productos category", "product category"}
    for c in cols:
        if _norm(c) in exact:
            return c
    for c in cols:
        cn = _norm(c)
        if "category" in cn and "sub" not in cn:
            return c
    return None

def find_subcategory_col(df: pd.DataFrame):
    cols = df.columns.tolist()
    exact = {"sub category", "sub-category", "subcategory", "productos sub category", "product sub category"}
    for c in cols:
        if _norm(c) in exact:
            return c
    for c in cols:
        cn = _norm(c)
        if ("subcategory" in cn) or ("sub" in cn and "category" in cn):
            return c
    return None

def mode_or_first(x: pd.Series):
    m = x.mode()
    return m.iloc[0] if len(m) else x.iloc[0]

# ============================================================
# 1) Load
# ============================================================
sales = pd.read_excel(FILES["sales"])
returns = pd.read_excel(FILES["returns"])
customer = pd.read_excel(FILES["customer"])
rm = pd.read_excel(FILES["rm"])
product = pd.read_excel(FILES["product"])

# ============================================================
# 2) Clean minimal
# ============================================================
customer = customer.drop_duplicates(subset=[COL["customer_id"]]).copy()

product = (product.sort_values([COL["product_id"]] + (["Product_Name"] if "Product_Name" in product.columns else []))
           .drop_duplicates(subset=[COL["product_id"]], keep="first")
           .copy())

# Detect category/subcategory on Product and normalize names
prod_cat_col = find_category_col(product)
prod_subcat_col = find_subcategory_col(product)

if prod_cat_col is not None and prod_subcat_col is not None and prod_cat_col == prod_subcat_col:
    prod_subcat_col = prod_cat_col
    for c in product.columns:
        cn = _norm(c)
        if c != prod_subcat_col and ("category" in cn and "sub" not in cn):
            prod_cat_col = c
            break

rename_map = {}
if prod_cat_col is not None:
    rename_map[prod_cat_col] = COL["category"]
if prod_subcat_col is not None:
    rename_map[prod_subcat_col] = COL["subcat"]
if rename_map:
    product = product.rename(columns=rename_map)

# Guarantee columns exist
if COL["category"] not in product.columns:
    product[COL["category"]] = np.nan
if COL["subcat"] not in product.columns:
    product[COL["subcat"]] = np.nan

# Sales types
sales[COL["order_date"]] = pd.to_datetime(sales[COL["order_date"]], errors="coerce")
for c in [COL["sales"], COL["profit"], COL["discount"], COL["qty"]]:
    if c in sales.columns:
        sales[c] = pd.to_numeric(sales[c], errors="coerce")

sales["Discount_clean"] = np.where(
    sales[COL["discount"]] > 1,
    sales[COL["discount"]] / 100,
    sales[COL["discount"]]
)

# Returns label
ret_order = (returns.groupby(COL["order_id"], as_index=False)
             .agg(productos_devueltos=(COL["returned_units"], "sum")))
ret_order["returned_flag"] = (ret_order["productos_devueltos"] > 0).astype(int)

# ============================================================
# 3) Build transactional df
# ============================================================
df = sales.merge(
    product[[COL["product_id"], COL["category"], COL["subcat"]]],
    on=COL["product_id"], how="left"
)

df = df.merge(customer, on=COL["customer_id"], how="left")
df = df.merge(rm, on=COL["region"], how="left")
df = df.merge(ret_order[[COL["order_id"], "returned_flag", "productos_devueltos"]],
              on=COL["order_id"], how="left")

df["returned_flag"] = df["returned_flag"].fillna(0).astype(int)
df["productos_devueltos"] = df["productos_devueltos"].fillna(0)

# Guarantee after merge
if COL["category"] not in df.columns:
    df[COL["category"]] = np.nan
if COL["subcat"] not in df.columns:
    df[COL["subcat"]] = np.nan

# ============================================================
# 4) Order-level dataset (1 fila por Order_ID)
# ============================================================
order_tbl = (df.groupby([COL["order_id"], COL["customer_id"]], as_index=False)
             .agg(
                 order_date=(COL["order_date"], "min"),
                 order_sales=(COL["sales"], "sum"),
                 order_profit=(COL["profit"], "sum"),
                 order_qty=(COL["qty"], "sum"),
                 discount_mean=("Discount_clean", "mean"),
                 discount_max=("Discount_clean", "max"),
                 n_lines=(COL["product_id"], "count"),
                 n_products=(COL["product_id"], "nunique"),
                 cat_div=(COL["category"], "nunique"),
                 subcat_div=(COL["subcat"], "nunique"),
                 ship_mode=(COL["ship_mode"], mode_or_first),
                 region=(COL["region"], mode_or_first),
                 country=(COL["country"], mode_or_first),
                 returned_flag=("returned_flag", "max")
             ))

# Time features
order_tbl["order_month"] = order_tbl["order_date"].dt.month
order_tbl["order_dow"] = order_tbl["order_date"].dt.dayofweek
order_tbl["is_weekend"] = (order_tbl["order_dow"] >= 5).astype(int)

# ============================================================
# 5) Customer historical features (SIN leakage)
# ============================================================
order_tbl = order_tbl.sort_values([COL["customer_id"], "order_date"]).reset_index(drop=True)
g = order_tbl.groupby(COL["customer_id"], group_keys=False)

order_tbl["prior_orders"] = g.cumcount()
order_tbl["prior_returns"] = g["returned_flag"].apply(lambda s: s.shift().fillna(0).cumsum())

order_tbl["prior_return_rate"] = np.where(
    order_tbl["prior_orders"] > 0,
    order_tbl["prior_returns"] / order_tbl["prior_orders"],
    0.0
)

order_tbl["prior_avg_discount"] = g["discount_mean"].apply(lambda s: s.shift().expanding().mean()).fillna(0.0)
order_tbl["prior_avg_order_sales"] = g["order_sales"].apply(lambda s: s.shift().expanding().mean())
order_tbl["prior_avg_order_sales"] = order_tbl["prior_avg_order_sales"].fillna(order_tbl["order_sales"].median())

order_tbl["prev_order_date"] = g["order_date"].shift(1)
order_tbl["recency_at_order_days"] = (order_tbl["order_date"] - order_tbl["prev_order_date"]).dt.days
order_tbl["recency_at_order_days"] = order_tbl["recency_at_order_days"].fillna(order_tbl["recency_at_order_days"].median())
order_tbl.drop(columns=["prev_order_date"], inplace=True)

# ============================================================
# 6) Add customer demographics (y forzar numérico donde aplique)
# ============================================================
demo_cols = [
    "monthly_salary",
    "age",
    "marital_status [1:married/ 0:Single]",
    "credit score"
]
keep_demo = [COL["customer_id"]] + [c for c in demo_cols if c in customer.columns]
order_tbl = order_tbl.merge(customer[keep_demo], on=COL["customer_id"], how="left")

# >>> FIX adicional: Coerción numérica para demografía (por 'x', 'N/A', etc.)
for dc in demo_cols:
    if dc in order_tbl.columns:
        order_tbl[dc] = pd.to_numeric(order_tbl[dc], errors="coerce")

# ============================================================
# 7) Train/Test split temporal (holdout)
# ============================================================
order_tbl = order_tbl.sort_values("order_date").reset_index(drop=True)

y = order_tbl["returned_flag"].astype(int)
X = order_tbl.drop(columns=["returned_flag"])

cutoff = X["order_date"].quantile(0.80)
train_idx = X["order_date"] <= cutoff
test_idx = X["order_date"] > cutoff

X_train = X.loc[train_idx].copy()
y_train = y.loc[train_idx].copy()
X_test = X.loc[test_idx].copy()
y_test = y.loc[test_idx].copy()

# Drop raw date
X_train = X_train.drop(columns=["order_date"])
X_test = X_test.drop(columns=["order_date"])

# ============================================================
# 8) Preprocess
# ============================================================
num_features = [
    "order_sales", "order_profit", "order_qty",
    "discount_mean", "discount_max",
    "n_lines", "n_products", "cat_div", "subcat_div",
    "order_month", "order_dow", "is_weekend",
    "prior_orders", "prior_returns", "prior_return_rate",
    "prior_avg_discount", "prior_avg_order_sales",
    "recency_at_order_days",
] + [c for c in demo_cols if c in X_train.columns]

cat_features = ["ship_mode", "region", "country"]

# >>> FIX CRÍTICO: forzar numérico en todas las features numéricas ANTES del pipeline
for c in num_features:
    if c in X_train.columns:
        X_train[c] = pd.to_numeric(X_train[c], errors="coerce")
    if c in X_test.columns:
        X_test[c] = pd.to_numeric(X_test[c], errors="coerce")

# (Opcional) Diagnóstico: identificar cuáles columnas traían strings
bad_cols = []
for c in num_features:
    if c in X_train.columns and X_train[c].dtype == "object":
        s0 = X_train[c]
        s1 = pd.to_numeric(s0, errors="coerce")
        if s0.notna().sum() > s1.notna().sum():
            bad_cols.append(c)
if bad_cols:
    print("Aviso: columnas numéricas tenían strings y fueron convertidas a NaN:", bad_cols)

numeric_pipe = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", RobustScaler())
])

categorical_pipe = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("ohe", safe_ohe())
])

preprocess = ColumnTransformer(
    transformers=[
        ("num", numeric_pipe, num_features),
        ("cat", categorical_pipe, cat_features)
    ],
    remainder="drop"
)

# ============================================================
# 9) Modelos: baseline + tuning
# ============================================================
logit = LogisticRegression(max_iter=2000, class_weight="balanced", solver="liblinear")
pipe_logit = Pipeline(steps=[("prep", preprocess), ("model", logit)])
pipe_logit.fit(X_train, y_train)
p_logit = pipe_logit.predict_proba(X_test)[:, 1]

hgb = HistGradientBoostingClassifier(random_state=RANDOM_STATE)
pipe_hgb = Pipeline(steps=[("prep", preprocess), ("model", hgb)])

tscv = TimeSeriesSplit(n_splits=5)
param_dist = {
    "model__learning_rate": [0.02, 0.05, 0.08, 0.10],
    "model__max_depth": [3, 5, 7, None],
    "model__max_leaf_nodes": [15, 31, 63],
    "model__min_samples_leaf": [20, 50, 100],
    "model__l2_regularization": [0.0, 0.1, 1.0],
}

search = RandomizedSearchCV(
    pipe_hgb,
    param_distributions=param_dist,
    n_iter=25,
    scoring="average_precision",
    cv=tscv,
    random_state=RANDOM_STATE,
    n_jobs=-1,
    refit=True
)
search.fit(X_train, y_train)

best_hgb = search.best_estimator_
p_hgb = best_hgb.predict_proba(X_test)[:, 1]

# ============================================================
# 10) Evaluación + umbral top10%
# ============================================================
def report_metrics(y_true, proba, name="model", threshold=0.5):
    pred = (proba >= threshold).astype(int)
    return {
        "model": name,
        "ROC_AUC": roc_auc_score(y_true, proba),
        "PR_AUC": average_precision_score(y_true, proba),
        "Precision@thr": precision_score(y_true, pred, zero_division=0),
        "Recall@thr": recall_score(y_true, pred, zero_division=0),
        "F1@thr": f1_score(y_true, pred, zero_division=0),
        "ConfusionMatrix": confusion_matrix(y_true, pred).tolist()
    }

def threshold_by_top_pct(proba, top_pct=0.10):
    return float(np.quantile(proba, 1 - top_pct))

def recall_at_top_pct(y_true, proba, top_pct=0.10):
    thr = threshold_by_top_pct(proba, top_pct)
    pred = (proba >= thr).astype(int)
    return recall_score(y_true, pred, zero_division=0), precision_score(y_true, pred, zero_division=0), thr

rec_logit_10, prec_logit_10, thr_logit_10 = recall_at_top_pct(y_test, p_logit, 0.10)
rec_hgb_10, prec_hgb_10, thr_hgb_10 = recall_at_top_pct(y_test, p_hgb, 0.10)

print("\n=== Baseline Logistic ===")
print(report_metrics(y_test, p_logit, "Logistic", threshold=thr_logit_10))
print("Top10% -> Recall:", rec_logit_10, "Precision:", prec_logit_10, "Threshold:", thr_logit_10)

print("\n=== Tuned HGB (best) ===")
print("Best params:", search.best_params_)
print(report_metrics(y_test, p_hgb, "HGB_tuned", threshold=thr_hgb_10))
print("Top10% -> Recall:", rec_hgb_10, "Precision:", prec_hgb_10, "Threshold:", thr_hgb_10)

print("\nClassification report (HGB_tuned @Top10% threshold):")
print(classification_report(y_test, (p_hgb >= thr_hgb_10).astype(int), zero_division=0))

# Lift por deciles (HGB)
tmp = pd.DataFrame({"y": y_test.values, "p": p_hgb})
tmp["decile"] = pd.qcut(tmp["p"], 10, labels=False, duplicates="drop")
lift = (tmp.groupby("decile")
          .agg(n=("y","size"), returns=("y","sum"), return_rate=("y","mean"))
          .reset_index()
          .sort_values("decile", ascending=False))
lift["cum_returns"] = lift["returns"].cumsum()
lift["cum_return_share"] = lift["cum_returns"] / lift["returns"].sum()

print("\nLift (deciles, HGB):")
print(lift)

# ============================================================
# 11) Exportables
# ============================================================
pred_out = X.loc[test_idx, [COL["order_id"], COL["customer_id"], "order_sales", "discount_mean", "ship_mode", "region"]].copy()
pred_out["y_true_returned"] = y_test.values
pred_out["p_return_hgb"] = p_hgb
pred_out["flag_top10pct"] = (p_hgb >= thr_hgb_10).astype(int)

pred_out.to_csv("order_return_predictions.csv", index=False)
lift.to_csv("lift_deciles_hgb.csv", index=False)

print("\nGuardados:")
print("- order_return_predictions.csv")
print("- lift_deciles_hgb.csv")



=== Baseline Logistic ===
{'model': 'Logistic', 'ROC_AUC': 0.7500490324267781, 'PR_AUC': 0.2051858865230908, 'Precision@thr': 0.22549019607843138, 'Recall@thr': 0.359375, 'F1@thr': 0.27710843373493976, 'ConfusionMatrix': [[877, 79], [41, 23]]}
Top10% -> Recall: 0.359375 Precision: 0.22549019607843138 Threshold: 0.6866573672354702

=== Tuned HGB (best) ===
Best params: {'model__min_samples_leaf': 100, 'model__max_leaf_nodes': 15, 'model__max_depth': None, 'model__learning_rate': 0.02, 'model__l2_regularization': 0.1}
{'model': 'HGB_tuned', 'ROC_AUC': 0.7439199790794979, 'PR_AUC': 0.2218842643352569, 'Precision@thr': 0.21568627450980393, 'Recall@thr': 0.34375, 'F1@thr': 0.26506024096385544, 'ConfusionMatrix': [[876, 80], [42, 22]]}
Top10% -> Recall: 0.34375 Precision: 0.21568627450980393 Threshold: 0.122986843537903

Classification report (HGB_tuned @Top10% threshold):
              precision    recall  f1-score   support

           0       0.95      0.92      0.93       956
          